In [18]:
import torch
import math
import matplotlib
import pandas as pd
from functools import reduce

percapin = pd.read_csv("../dataset/Percapita-Income - Sheet1.csv",usecols=[1,2])
total = pd.read_csv("../dataset/TotalCases,TotalDeaths,TotalRecovered - Sheet1.csv",usecols=[1,7,9,10,13])
vaccine = pd.read_csv("../dataset/vaccine_data_global.csv",usecols=[0,3,4])
gdp = pd.read_csv("../dataset/WEO_Data.csv", encoding='latin-1',usecols=[0,6])
#print(total.head(5))
#fatality = pd.read_csv("../dataset/04-16-2021.csv")

data_frames = [total,percapin,vaccine,gdp]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Country'],
                                            how='inner'), data_frames)
#df_merged['vaccination_rate'] = df_merged['People_fully_vaccinated']/df_merged['Population'].str.replace(',', '').astype(float)              
#print(gdp.head(5))
#print(total.head(5))
#output = pd.merge(percapin,total, on='Country',how='inner')
print(df_merged)
pd.DataFrame.to_csv(df_merged, '../dataset/merged.csv', sep=',', na_rep='.', index=False)


                                               Country Active\nCases  \
0                                                World    23,810,939   
1                                        United States     6,862,240   
2                                                India     1,200,979   
3                                               Brazil     1,248,447   
4                                               France     4,650,143   
..                                                 ...           ...   
265                     St. Vincent and the Grenadines           NaN   
266                           Taiwan Province of China           NaN   
267                                 West Bank and Gaza           NaN   
268                                                NaN           NaN   
269  International Monetary Fund, World Economic Ou...           NaN   

    Tot Cases/\n1M pop Deaths/\n1M pop     Population     US$  \
0               17,530           378.4            NaN     NaN   
1    